<a href="https://colab.research.google.com/github/Jenny16x/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://security.ubuntu.com/ubuntu bionic-security/multiverse amd64 Packages [26.8 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:1

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-07 22:16:57--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.67MB/s    in 0.2s    

2022-01-07 22:16:57 (5.67 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://challenge16-bucket.s3.us-west-1.amazonaws.com/amazon_reviews_us_Kitchen_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id" : "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   23042837|             1|
|   48875707|             2|
|   48088660|             1|
|   20781887|             6|
|    1075003|             1|
|   28377689|             3|
|    8338749|             1|
|   39271457|             6|
|   38209321|             5|
|   51397605|             1|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004NWK4W0|Wildkin Pink Leop...|
|B00IE70YFC|Wilton 415-2178 3...|
|B00004W499|George Foreman GG...|
|B000Q945FG|BIA Cordon Bleu B...|
|B007WTOIPA|Clorox Sales Co B...|
|B006IC09P0|Utenlid Acrylic R...|
|B00CDSU2MI|2000 Diamond Tabl...|
|B000WJMTNA|OXO Good Grips St...|
|B00DYRI13O|Reed & Barton 148...|
|B005FPD67Y|     Ergo Chef Tongs|
+----------+--------------------+
only showing top 10 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3DT59XH7HXR9K|   37000337|B00303FI0G|     529320574| 2015-08-31|
|R1LFS11BNASSU8|   15272914|B00JCZKZN6|     274237558| 2015-08-31|
|R296RT05AG0AF6|   36137863|B00JLIKA5C|     544675303| 2015-08-31|
|R3V37XDZ7ZCI3L|   43311049|B000GBNB8G|     491599489| 2015-08-31|
|R14GU232NQFYX2|   13763148|B00VJ5KX9S|     353790155| 2015-08-31|
| RZQH4V7L2O1PL|   19009420|B00HYB5YY0|     432241873| 2015-08-31|
|R1F8JMOSPJ3KO7|   40599388|B000HEBAV2|     584680984| 2015-08-31|
|R1ZISGY2BWW4Z5|   22719359|B0012DS4GG|     772637306| 2015-08-31|
|R17PW4I3AE5WZW|   47478640|B00FLQ4EE6|     755416578| 2015-08-31|
|R3D93G1KTP6A8P|   34195504|B00DBS9OTG|     648762742| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes","total_votes","vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3DT59XH7HXR9K|          5|            0|          0|   N|                Y|
|R1LFS11BNASSU8|          5|            0|          1|   N|                Y|
|R296RT05AG0AF6|          5|            0|          0|   N|                Y|
|R3V37XDZ7ZCI3L|          5|            0|          1|   N|                Y|
|R14GU232NQFYX2|          5|            0|          0|   N|                Y|
| RZQH4V7L2O1PL|          1|            1|          1|   N|                Y|
|R1F8JMOSPJ3KO7|          5|            0|          0|   N|                Y|
|R1ZISGY2BWW4Z5|          5|            0|          0|   N|                Y|
|R17PW4I3AE5WZW|          5|            0|          0|   N|                Y|
|R3D93G1KTP6A8P|          3|            0|          0|   N|     

In [ ]:
#Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20 
#to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

vote_count_df = df.filter(df.total_votes >=20)
vote_count_df.show(10, truncate= False)


+-----------+-----------+--------------+----------+--------------+---------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+----------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of 
#helpful_votes divided by total_votes is equal to or greater than 50%.

new_vote_count_df = vote_count_df.filter((vote_count_df.helpful_votes/vote_count_df.total_votes) >= 0.5)
new_vote_count_df.show(10, truncate= False)

+-----------+-----------+--------------+----------+--------------+---------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+----------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was 
#written as part of the Vine program (paid), vine == 'Y'.

vine_program_paid_count= new_vote_count_df.filter(new_vote_count_df.vine == 'Y')
vine_program_paid_count.show(10, truncate= False)

+-----------+-----------+--------------+----------+--------------+----------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.
vine_program_unpaid_count= new_vote_count_df.filter(new_vote_count_df.vine == 'N')
vine_program_unpaid_count.show(10, truncate= False)

+-----------+-----------+--------------+----------+--------------+---------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+----------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Determine 
#the total number of reviews, 
#the number of 5-star reviews, and 
#the percentage of 5-star reviews for the two types of review (paid vs unpaid).

paid_total_review = vine_program_paid_count.count()
paid_total_review 

1207

In [ ]:
paid_five_stars = vine_program_paid_count.filter(vine_program_paid_count.star_rating == 5).count()
paid_five_stars

509

In [ ]:
paid_five_stars_per = (paid_five_stars/paid_total_review)*100
paid_five_stars_per

42.17067108533554

In [ ]:
unpaid_total_review = vine_program_unpaid_count.count()
unpaid_total_review 

97839

In [ ]:
unpaid_five_stars = vine_program_unpaid_count.filter(vine_program_unpaid_count.star_rating == 5).count()
unpaid_five_stars

45858

In [ ]:
unpaid_five_stars_per = (unpaid_five_stars/unpaid_total_review)*100
unpaid_five_stars_per

46.87087971054488